# Advent of Code 2023
## Day 21
*<https://adventofcode.com/2023/day/21>*

In [13]:
import heapq
import math
import re
import functools as ft
from collections import Counter, defaultdict, deque, namedtuple
from itertools import combinations, permutations, product
from string import ascii_letters, ascii_lowercase, ascii_uppercase

import IPython
import z3
from rich import inspect, pretty, print

from new_helper import *

pretty.install()

In [14]:
DAY = 21
input_str = get_aoc_input(DAY, 2023)
part_1 = part_2 = 0

In [15]:
inp = input_str.parse_grid()

In [16]:
sx, sy = inp.find("S")


def is_garden(x: int, y: int):
    return inp[x, y] == "." or inp[x, y] == "S"


def num_reachable(sx: int, sy: int, N: int):
    visited = set()
    targets = set()
    Q: deque[tuple[int, int, int]] = deque()
    Q.append((0, sx, sy))

    while Q:
        d, x, y = Q.popleft()

        if d % 2 == N % 2:
            targets.add((x, y))

        if d >= N:
            continue

        if (x, y) in visited:
            continue

        visited.add((x, y))

        for nx, ny in inp.adjacent_4_positions(x, y):
            if is_garden(nx, ny):
                Q.append((d + 1, nx, ny))

    return len(targets)


part_1 = num_reachable(sx, sy, 64)

Assumes that the borders of each grid, as well the starting row/column is completely open, and that the starting position is in the exact center.

In [17]:
N = 26501365

assert inp.width == inp.height

grid_size = inp.width
assert grid_size % 2 == 1

diamond_radius_grids = (N + 1) // grid_size
diamond_radius = (diamond_radius_grids - 1) * grid_size + math.ceil(grid_size / 2)

num_start_grid = 1 + sum(4 * i for i in range(2, diamond_radius_grids, 2))
num_alt_grid = sum(4 * i for i in range(1, diamond_radius_grids, 2))

start_grid_covered = num_reachable(sx, sy, 1000 + N % 2)
alt_grid_covered = num_reachable(sx, sy, 1000 + (N + 1) % 2)

inner_coverage = num_start_grid * start_grid_covered + num_alt_grid * alt_grid_covered
part_2 += inner_coverage

center_reach = N - diamond_radius
for x, y in (
    (sx, inp.height - 1),
    (0, sy),
    (sx, 0),
    (inp.width - 1, sy),
):
    part_2 += num_reachable(x, y, center_reach)

corner_1_reach = N - (diamond_radius + math.ceil(grid_size / 2))
corner_2_reach = N - (diamond_radius - grid_size + math.ceil(grid_size / 2))
for x, y in (
    (0, inp.height - 1),
    (0, 0),
    (inp.width - 1, 0),
    (inp.width - 1, inp.height - 1),
):
    if corner_1_reach >= 0:
        part_2 += num_reachable(x, y, corner_1_reach) * diamond_radius_grids

    part_2 += num_reachable(x, y, corner_2_reach) * (diamond_radius_grids - 1)

In [18]:
print_part_1(part_1)
print_part_2(part_2)

Part 1: 3858

Part 2: 636350496972143